<div style="width: 38.5%;">
    <p><strong>City College of San Francisco</strong><p>
    <hr>
    <p>MATH 108 - Foundations of Data Science</p>
</div>

# Lecture 33: Linear Regression

Associated Textbook Sections: [15.2](https://inferentialthinking.com/chapters/15/2/Regression_Line.html)

## Outline

* [Prediction](#Prediction)
* [Linear Regression](#Linear-Regression)
* [Slope & Intercept](#Slope-&-Intercept)

## Set Up the Notebook

In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Some functions for plotting. You are not required to understand how these work.
def resize_window(lim=3.5):
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)

def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope + intercept
    plt.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plt.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_scatter(r):
    """Generate a scatter plot with a correlation approximately r"""
    plt.figure(figsize=(5,5))
    x, y = make_correlated_data(r)
    plt.scatter(x, y, color='darkblue', s=20)
    plt.xlim(-4, 4)
    plt.ylim(-4, 4)
    
def r_table(r):
    """
    Generate a table of 1000 data points with a correlation approximately r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

def nn_prediction_galton(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.5 inches.
    """
    neighbors = heights.where(
        'MidParent', are.between(h - 0.5, h + 0.5))
    return np.mean(neighbors.column('Child'))

---

## Prediction

In [ ]:
galton = Table.read_table('data/galton.csv')
heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight'))

### Galton's Heights (`child` vs. `midParent`)

In [ ]:
heights.scatter('MidParent')

* Oval shaped
* Moderate positive correlation
* How can we predict child height from mid-parent height?

### Galton's Heights (within 0.5 of `midParentHeight`)


In [ ]:
heights.scatter('MidParent')
plt.plot([68-0.5, 68-0.5], [55, 80], color="red", lw=4)
plt.plot([68+0.5, 68+0.5], [55, 80], color="red", lw=4)
plt.scatter(68, nn_prediction_galton(68), color="gold", s=60, zorder=3)
plt.show()

### Galton's Heights (predictions)


In [ ]:
predicted_child = heights.apply(nn_prediction_galton, 'MidParent')
heights_with_predictions = heights.with_column('Predicted Child', predicted_child)
heights_with_predictions.scatter('MidParent')

### Nearest Neighbor Regression

* A method for prediction:
    * Group each x with similar (nearby) x values
    * Average the corresponding y values for each group
    * For each x value, the prediction is the average of the y values in its nearby group.
* The graph of these predictions is the "graph of averages".
* If the association between x and y is linear, then points in the graph of averages tend to fall on a line.


### Where is the prediction line? (`r=0.99`)


In [ ]:
r_scatter(0.99)

### Where is the prediction line? (`r=0`)


In [ ]:
r_scatter(0)

### Demo: Prediction lines

Identify the best fit line in relation to the nearest neighbor predictions with data showing an r value of 0.99.

In [ ]:
example = r_table(0.99)
example.show(3)

In [ ]:
example.scatter('x', 'y')

In [ ]:
example.scatter('x', 'y')
resize_window()

In [ ]:
def nn_prediction_example(x_val):
    """ Predicts y-value for x based on the example table """
    neighbors = example.where('x', are.between(x_val - .25, x_val + .25))
    return np.mean(neighbors.column('y'))
    

In [ ]:
nn_prediction_example(-2.25)

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example.scatter('x')
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1, color='dodgerblue')
resize_window()

Identify the best fit line in relation to the nearest neighbor predictions with data showing an r value of 0.

In [ ]:
example = r_table(0)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example = example.with_column(
    'Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_line(slope = 0)
resize_window()

Identify the best fit line in relation to the nearest neighbor predictions with data showing an r value of 0.5.

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()
draw_vertical_line(1.5)
draw_line(slope=1, intercept=0)

In [ ]:
example = example.with_column('Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_line(slope=1)
draw_vertical_line(1.5)
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1, intercept=0, color='red')
draw_line(slope=0.5, intercept=0, color='dodgerblue')
resize_window()

---

## Linear Regression

### Linear Regression

* A statement about x and y pairs
    * Measured in standard units
    * Describing the deviation of $x$ from 0 (the average of $x$'s)
    * And the deviation of $y$ from 0 (the average of $y$'s)
* On average, $y$ deviates from 0 less than $x$ deviates from 0
* Regression Line Equation: $y_{\text{su}} = r \times x_{\text{su}}$
* Not true for all points — a statement about averages

---

## Slope & Intercept

### Regression Line Equation

* In original units, the regression line has this equation: $$\frac{\text{estimate of } y - \text{average of } y}{\text{SD of } y} = r \times \frac{\text{the given } x - \text{average of } x}{\text{SD of } x}$$
* Lines can be expressed by slope & intercept as: $y = \text{slope} \times x + \text{intercept}$

### Regression Line

<img src="img/lec30_regression_line.png" width=60%>

### Slope and Intercept

* `estimate_of_y = slope * x + intercept`
* `slope = r * (SD_y / SD_x)`
* `intercept = mean_y - slope * mean_x`

### Demo: Defining the line

Use the `standard_units` and `correlation` functions to define functions `slope` and `intercept`. 

In [ ]:
# Copy-pasted from above
def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)

In [ ]:
def slope(t, x, y):
    return ...

def intercept(t, x, y):
    return ... 

In [ ]:
example = r_table(0.5)
slope(example, 'x', 'y'), intercept(example, 'x', 'y')

### Units

Suppose we use linear regression to predict candy prices (in dollars) from sugar content (in grams). The units of the following are:

* $r$ - no units
* The slope - dollars/grams
* The intercept - dollars

### Example

* A course has a midterm (average 70; standard deviation 10)
and a really hard final (average 50; standard deviation 12)
* If the scatter diagram comparing midterm & final scores for students has an oval shape with correlation 0.75, then... 
    1. What do you expect the average final score would be for students who scored 90 on the midterm?
    2. How about 60 on the midterm?

In [ ]:
def predict(midterm_score):
    ...

In [ ]:
# 1.
predict(90)

In [ ]:
# 2.
predict(60)

---

<footer>
    <p>Adopted from UC Berkeley DATA 8 course materials.</p>
    <p>This content is offered under a <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/">CC Attribution Non-Commercial Share Alike</a> license.</p>
</footer>